<a href="https://colab.research.google.com/github/sergioGarcia91/ML_Carolina_Bays/blob/main/01_Dataset_generation_AOI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook, each Landsat 8 L2SP image will be cropped to match the Area of Interest (AOI). All bands will be preserved except for the thermal band. Additionally, a binary pixel raster (Y) will be considered, where pixel values will be adjusted to mask shadows and clouds. A value of 1 represents areas with Fairy Circles (FC), while 0 indicates areas without it.

# Start

In [ ]:
# Install required libraries
!pip install rasterio

In [ ]:
import numpy as np
import rasterio
import os
import time
import geopandas as gpd
import tarfile
import tempfile
import shutil

from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.mask import mask
from rasterio.transform import from_bounds
from rasterio.merge import merge
from pyproj import CRS
from IPython.display import clear_output

In [ ]:
# Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Functions

## temp_folder

In [ ]:
def temp_folder():
  # Create a temporary directory to store the extracted files .. and others
  return tempfile.mkdtemp()

## extract_tar

In [ ]:
def extract_tar(path_tar, temp_folder):
  # Check if the .tar file exists
  if not os.path.exists(path_tar):
    print(f"The file {path_tar} does not exist.")
    return None

  # Open the .tar file and extract all its contents into the temporary folder
  with tarfile.open(path_tar, 'r') as tar_file:
    tar_file.extractall(path=temp_folder)

  print(f"Files extracted to the temporary folder: {temp_folder}")
  return temp_folder

## delete_temporal_folder

In [ ]:
def delete_temporal_folder(temporal_path):
  # Check if the temporary folder exists and delete it
  if os.path.exists(temporal_path):
    try:
      shutil.rmtree(temporal_path)
      print(f"The temporary folder {temporal_path} has been deleted.")
    except PermissionError:
      print(f"Error: You do not have permission to delete the folder {temporal_path}.")
  else:  # If the folder has already been deleted or does not exist
    print(f"The temporary folder {temporal_path} does not exist or has already been deleted.")


## scene_id

In [ ]:
def scene_id(tarFile):
  # Scene ID, which precedes all L8L2 files up to the _T1 expression.
  return tarFile[:-4]

## load_TIFs

In [ ]:
def load_TIFs(scene_id, folder_path):
  tiff_files = []
  for filename in os.listdir(folder_path):
    if (scene_id in filename) and (filename.endswith('.TIF')):
      # Guardar nombre del archivo TIFF
      tiff_files.append(filename)
  return tiff_files

## scaleImage

In [ ]:
def scaleImage(tiff_files, folder_path, path_shape, folder_temp_save, AOIname=''):
  scaledFiles = []
  # Read the shapefile with geopandas
  gdf = gpd.read_file(path_shape)
  # Get the coordinate reference system (CRS) of the shapefile
  shapefile_crs = gdf.crs

  for filename in tiff_files:
    file_path = os.path.join(folder_path, filename)
    band_name = filename.split('_')[-1].replace('.TIF', '')
    with rasterio.open(file_path) as src:
      raster_crs = CRS(src.crs)
      # Convert CRS if needed
      if shapefile_crs != raster_crs:
        gdf = gdf.to_crs(raster_crs.to_string())
      # Convert the shapefile geometry to a list of features
      geoms = gdf.geometry.values

      if band_name in ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']:
        band_, out_transform = rasterio.mask.mask(src, geoms, crop=True)
        profile = src.meta.copy()
        # Scaling the band
        band_ = band_[0] * 0.0000275 - 0.2
        # Define a valid range for the band
        valid_min = 0 #- 0.2
        # Replace values outside the valid range with 0
        band_ = np.where((band_ <= valid_min), 0, band_)
        band_ = np.where((band_ >= 1), 1, band_)
        # Replace nan values with 0
        band_ = np.where((band_ == np.nan), 0, band_)
        # Update the profile
        profile.update({"driver": "GTiff",
                        "height": band_.shape[0],
                        "width": band_.shape[1],
                        "transform": out_transform,
                        "dtype": "float32"})
        # Save the scaled band into a single TIFF file
        newFileName = f'{AOIname}_{filename[16:25]}_{band_name}.TIF'
        print(newFileName)
        scaledFiles.append(newFileName)
        with rasterio.open(folder_temp_save + '/' + newFileName, 'w', **profile) as dst:
          dst.write(band_, 1)

      elif band_name == 'CDIST':
        band_, out_transform = rasterio.mask.mask(src, geoms, crop=True)
        profile = src.meta.copy()
        # Scaling the band
        band_ = band_[0] * 0.01 # In kilometers
        # Define a valid range for the band
        valid_min = -99.99 # Taken when evaluating the band in QGIS
        # Replace values outside the valid range with np.nan
        band_ = np.where((band_ <= valid_min), np.nan, band_)
        # Update the profile
        profile.update({"driver": "GTiff",
                        "height": band_.shape[0],
                        "width": band_.shape[1],
                        "transform": out_transform,
                        "dtype": "float32"})
        # Save the scaled band into a single TIFF file
        newFileName = f'{AOIname}_{filename[16:25]}_CDIST.TIF'
        print(newFileName)
        scaledFiles.append(newFileName)
        with rasterio.open(folder_temp_save + '/' + newFileName, 'w', **profile) as dst:
          dst.write(band_, 1)

      elif band_name == 'PIXEL':
        band_, out_transform = rasterio.mask.mask(src, geoms, crop=True)
        profile = src.meta.copy()
        # Scaling the band
        band_ = band_[0] * 1 # This band is binary
        # Update the profile
        profile.update({"driver": "GTiff",
                        "height": band_.shape[0],
                        "width": band_.shape[1],
                        "transform": out_transform,
                        "dtype": "float32"})
        # Save the scaled band into a single TIFF file
        newFileName = f'{AOIname}_{filename[16:25]}_QA_PIXEL.TIF'
        print(newFileName)
        scaledFiles.append(newFileName)
        with rasterio.open(folder_temp_save + '/' + newFileName, 'w', **profile) as dst:
          dst.write(band_, 1)

  return scaledFiles

## clip_Y

In [ ]:
def clip_Y(path_Y, scene_id, path_shape, folder_temp_save, AOIname=''):
  scaled_Y = []
  # Read the shapefile with geopandas
  gdf = gpd.read_file(path_shape)
  # Get the coordinate reference system (CRS) of the shapefile
  shapefile_crs = gdf.crs
  filename = scene_id
  file_path = path_Y
  band_name = 'Y'
  with rasterio.open(file_path) as src:
    raster_crs = CRS(src.crs)
    # Convert CRS if needed
    if shapefile_crs != raster_crs:
      gdf = gdf.to_crs(raster_crs.to_string())
    # Convert the shapefile geometry to a list of features
    geoms = gdf.geometry.values

    if band_name == 'Y':
      band_, out_transform = rasterio.mask.mask(src, geoms, crop=True)
      profile = src.meta.copy()
      # Scaling the band
      band_ = band_[0] * 1  # In kilometers
      # Update the profile
      profile.update({"driver": "GTiff",
                      "height": band_.shape[0],
                      "width": band_.shape[1],
                      "transform": out_transform,
                      "dtype": "float32"})
      # Save the scaled band into a single TIFF file
      newFileName = f'{AOIname}_{filename[17:25]}_Y.TIF'
      print(newFileName)
      scaled_Y.append(newFileName)
      with rasterio.open(folder_temp_save + '/' + newFileName, 'w', **profile) as dst:
        dst.write(band_, 1)

  return scaled_Y

## mask_clouds_shadows_Y

In [ ]:
def mask_clouds_shadows_Y(scaled_Y,
                          folder_TIFs_scaled,
                          maskCloudDistance=0.05, # in kilometers
                          ):
    for file_name in os.listdir(folder_TIFs_scaled):
      # Leer la banda _CDIST
      if 'CDIST' in file_name:
        CDIST_file = file_name
      elif 'QA_PIXEL' in file_name:
        QA_PIXEL_file = file_name

    print('CDIST file: ', CDIST_file)
    print('QA_PIXEL file: ', QA_PIXEL_file)

    with rasterio.open(folder_TIFs_scaled + '/' + CDIST_file) as cdist:
      cd_data = cdist.read(1)
    with rasterio.open(folder_TIFs_scaled + '/' + QA_PIXEL_file) as qaPixel:
      shadows = qaPixel.read(1)

    # Leer banda Y recortada
    path_scaled_Y = os.path.join(folder_TIFs_scaled, scaled_Y)
    with rasterio.open(path_scaled_Y) as src:
      band_Y_data = src.read(1)
      profile = src.profile

    # Reemplazar valores fuera del rango válido con 0
    band_Y_data = np.where((cd_data <= maskCloudDistance), 0, band_Y_data)
    # 23888 higth confidence cloud shadow
    band_Y_data = np.where((shadows == 23888), 0, band_Y_data)
    print('    Masking...')
    scaled_Y = scaled_Y[:-4] + '_mask.TIF'
    with rasterio.open(folder_TIFs_scaled + '/' + scaled_Y, 'w', **profile) as dst:
      dst.write(band_Y_data, 1)


## copy_files

In [ ]:
def copy_files(temp_folder, dest_folder):
    # Ensure the destination folder exists
    os.makedirs(dest_folder, exist_ok=True)

    # Get the list of files with the specified extension in the temporary folder
    files = os.listdir(temp_folder)

    # Copy each file to the destination folder
    for file_to_copy in files:
        src_path = os.path.join(temp_folder, file_to_copy)
        dest_path = os.path.join(dest_folder, file_to_copy)
        shutil.copy2(src_path, dest_path)  # copy2 preserves metadata

    print(f"Copied {len(files)} files from {temp_folder} to {dest_folder}")

## merge_bands

In [ ]:
def merge_bands(input_folder, scene_id, AOIname=''):
    band_files = sorted([os.path.join(input_folder, f)
                         for f in os.listdir(input_folder)
                         if f.endswith('.TIF') and any(band in f for band in ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7'])])

    if not band_files:
        print("No Landsat 8 bands found in the folder.")
        return

    # Open all bands and store their data
    bands = []
    meta = None
    for i, band_path in enumerate(band_files):
      with rasterio.open(band_path) as src:
        if i == 0:
          meta = src.meta.copy()  # Copy metadata from the first band
        bands.append(src.read(1))  # Read the first band from each file

    # Update metadata for multiband file
    meta.update(count=len(bands))  # Set number of bands

    # Write the multiband raster
    output_path = input_folder
    output_file = AOIname + scene_id[16:25] + '_merge.TIF'
    with rasterio.open(output_path + '/' + output_file, 'w', **meta) as dst:
        for idx, band in enumerate(bands, start=1):
            dst.write(band, idx)

    print(f"Multiband TIF saved as: {output_file}")


# Run

In [ ]:
path_tar_folder = '/content/drive/MyDrive/UIS/Doctorado_UIS2198589/1_semestre/TopicosAvanzadosGeofisica/FC_CarolinaBais/L8_L2_C2_T1'
tar_files = os.listdir(path_tar_folder)
print(tar_files)

raster_Y = '/content/drive/MyDrive/UIS/Doctorado_UIS2198589/1_semestre/TopicosAvanzadosGeofisica/FC_CarolinaBais/ProyectoSIG/Raster_Y.tif'

AOInameShape = 'AOI_04'

pathDrive = '/content/drive/MyDrive/UIS/Doctorado_UIS2198589/1_semestre/TopicosAvanzadosGeofisica/FC_CarolinaBais/Dataset_AOI'

['LC08_L2SP_016036_20170712_20200903_02_T1.tar', 'LC08_L2SP_016036_20160404_20200907_02_T1.tar', 'LC08_L2SP_016036_20160826_20200906_02_T1.tar', 'LC08_L2SP_016036_20200720_20210330_02_T1.tar', 'LC08_L2SP_016036_20140704_20200911_02_T1.tar', 'LC08_L2SP_016036_20150504_20201015_02_T1.tar', 'LC08_L2SP_016036_20230627_20230710_02_T1.tar', 'LC08_L2SP_016036_20240105_20240113_02_T1.tar', 'LC08_L2SP_016036_20201227_20210310_02_T1.tar', 'LC08_L2SP_016036_20211214_20211223_02_T1.tar', 'LC08_L2SP_016036_20241104_20241113_02_T1.tar', 'LC08_L2SP_016036_20180205_20200902_02_T1.tar', 'LC08_L2SP_016036_20201008_20201016_02_T1.tar', 'LC08_L2SP_016036_20151112_20200908_02_T1.tar', 'LC08_L2SP_016036_20180715_20200831_02_T1.tar', 'LC08_L2SP_016036_20161029_20200905_02_T1.tar', 'LC08_L2SP_016036_20191225_20200824_02_T1.tar', 'LC08_L2SP_016036_20140602_20200911_02_T1.tar', 'LC08_L2SP_016036_20140821_20200911_02_T1.tar', 'LC08_L2SP_016036_20150317_20200909_02_T1.tar', 'LC08_L2SP_016036_20170202_20200905_02_

In [ ]:
totalTarFiles = len(tar_files)
countTarFiles = 1
# 105 tif ... 2 generados ... un total de 210 por AOI
# 840 tif en total

for file_tar in tar_files[:]:
  print(f'File {countTarFiles} of {totalTarFiles}')
  countTarFiles += 1
  temp_folder_tar = temp_folder()
  #print(temp_folder_tar)

  temp_folder_tif = temp_folder()
  #print(temp_folder_tif)

  path_tar_file = path_tar_folder + '/' + file_tar
  #print(path_tar_file)

  extract_tar(path_tar_file, temp_folder_tar)

  scene_id_ = scene_id(file_tar)
  #print(scene_id_)

  list_TIF_files = load_TIFs(scene_id_, temp_folder_tar)

  path_shape_AOI = f"/content/drive/MyDrive/UIS/Doctorado_UIS2198589/1_semestre/TopicosAvanzadosGeofisica/FC_CarolinaBais/ProyectoSIG/CarolinaBays_{AOInameShape}.shp"

  scaleImage(tiff_files = list_TIF_files,
             folder_path = temp_folder_tar,
             path_shape = path_shape_AOI,
             folder_temp_save = temp_folder_tif,
             AOIname = AOInameShape)

  Y_file_clip = clip_Y(path_Y = raster_Y,
                      scene_id = scene_id_,
                      path_shape = path_shape_AOI,
                      folder_temp_save = temp_folder_tif,
                      AOIname = AOInameShape)

  #os.listdir(temp_folder_tif)

  mask_clouds_shadows_Y(scaled_Y = Y_file_clip[0],
                        folder_TIFs_scaled = temp_folder_tif,
                        maskCloudDistance = 0.05 # in kilometers
                        )

  #os.listdir(temp_folder_tif)
  merge_bands(input_folder = temp_folder_tif,
              scene_id = scene_id_,
              AOIname = AOInameShape)

  for file_to_del in os.listdir(temp_folder_tif):
    if not (('merge' in file_to_del) or ('mask' in file_to_del)):
      os.remove(temp_folder_tif + '/' + file_to_del)

  #os.listdir(temp_folder_tif)

  copy_files(temp_folder = temp_folder_tif,
           dest_folder = pathDrive)

  delete_temporal_folder(temp_folder_tar)
  delete_temporal_folder(temp_folder_tif)

  print('\n')
  clear_output()
  #print(os.listdir(pathDrive))

# End